In [138]:
import pandas as pd
import numpy as np
from geopy import distance

In [139]:
# read flight record CSV files
alaskan = pd.read_csv("./raw/alaskan.csv")
american = pd.read_csv("./raw/american.csv")
delta = pd.read_csv("./raw/delta.csv")
southwest = pd.read_csv("./raw/southwest.csv")
united = pd.read_csv("./raw/united.csv")

# concatenate files
record = pd.concat([alaskan, american, delta, southwest, united])

# convert airline code to airline name
code_to_name = {
            "AA": "American",
            "AS": "Alaskan",
            "DL": "Delta",
            "UA": "United",
            "WN": "Southwest"
        }
record["Airline"] = record["Carrier Code"].apply(lambda code: code_to_name[code])
record = record.drop("Carrier Code", axis=1)

# change feature names
record = record.rename(columns={
    "Date (MM/DD/YYYY)": "Date"
    })

# extract day of the week
record["Day of the Week"] = pd.to_datetime(record["Date"]).dt.day_name()

# drop rows
record = record[record["Scheduled departure time"].apply(lambda x: int(x.split(":")[0]) < 23)].reset_index(drop=True)

In [140]:
# read airport CSV
airport = pd.read_csv("./raw/airport-codes.csv")
sandiego_coord = tuple(airport[airport["iata_code"]=="SAN"]["coordinates"].iloc[0].split(",")[::-1])

# keep only airports found in the record DF
airport_codes = record["Destination Airport"].unique()
airport = airport[airport["local_code"].isin(airport_codes)]

# extract data from columns
airport["state"] = airport["iso_region"].str.split("-").apply(lambda x: x[1])
airport["longitude"] = airport["coordinates"].str.split(",").apply(lambda x: x[0])
airport["latitude"] = airport["coordinates"].str.split(",").apply(lambda x: x[1])
airport["distance"] = list(pd.Series(zip(airport["latitude"], airport["longitude"]))\
    .apply(lambda coord: distance.distance(sandiego_coord, coord).km))

# keep only relevant columns
airport = airport[["name", "iso_country", "state", "local_code", "longitude", "latitude", "distance"]]
airport = airport.reset_index(drop=True)

In [141]:
# join datasets
combined = pd.merge(left=record, right=airport, how="left", left_on="Destination Airport", right_on="local_code")

# rename columns
combined = combined.rename(columns={
    "Destination Airport": "Destination Airport Code",
    "name": "Destination Airport Name", 
    "iso_country": "Destination Airport Country", 
    "state": "Destination Airport State", 
    "local_code": "Destination Airport Local Code", 
    "longitude": "Destination Airport Longitude",
    "latitude": "Destination Airport Latitude",
    "distance": "Destination Airport Distance"
    })

# add noise
N = combined.shape[0]
min_dist = combined["Destination Airport Distance"].min()
max_dist = combined["Destination Airport Distance"].max()
def add_noise(x):
    while True:
        dist = np.random.normal(x, x, 1)[0]
        if dist > min_dist and dist < max_dist:
            return dist
combined["Destination Airport Distance"] = combined["Destination Airport Distance"].apply(add_noise) 

# drop destination
combined = combined[combined["Destination Airport Country"]!="PG"].reset_index(drop=True)

# export csv
combined.to_csv("data.csv")

In [142]:
combined.head()

,Date,Flight Number,Tail Number,Destination Airport Code,Scheduled departure time,Actual departure time,Departure delay (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),...,Delay Late Aircraft Arrival (Minutes),Airline,Day of the Week,Destination Airport Name,Destination Airport Country,Destination Airport State,Destination Airport Local Code,Destination Airport Longitude,Destination Airport Latitude,Destination Airport Distance
0,01/01/2018,333,N560AS,PDX,06:30,06:35,5,0,0,81,...,0,Alaskan,Monday,Portland International Airport,US,OR,PDX,-122.5979996,45.58869934,2067.666357
1,01/01/2018,397,N282AK,SEA,17:10,17:01,-9,0,0,0,...,0,Alaskan,Monday,Seattle Tacoma International Airport,US,WA,SEA,-122.308998,47.449001,5077.565465
2,01/01/2018,405,N487AS,SEA,14:45,15:57,72,20,0,0,...,45,Alaskan,Monday,Seattle Tacoma International Airport,US,WA,SEA,-122.308998,47.449001,1506.832607
3,01/01/2018,471,N408AS,SEA,06:20,06:18,-2,0,0,0,...,0,Alaskan,Monday,Seattle Tacoma International Airport,US,WA,SEA,-122.308998,47.449001,2391.001749
4,01/01/2018,481,N319AS,SEA,09:56,12:46,170,0,0,0,...,154,Alaskan,Monday,Seattle Tacoma International Airport,US,WA,SEA,-122.308998,47.449001,3794.394263
